In [28]:
import os
from dotenv import load_dotenv
from rich import print
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [53]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002758AF23B10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002758AF106D0>, model_name='llama-3.1-8b-instant', groq_api_key=SecretStr('**********'))

In [54]:
from langchain_core.messages import HumanMessage

llm.invoke(
    [HumanMessage(content="I'm Aasher, a student of GenAI")]
)

AIMessage(content='Hello Aasher! Nice to meet you! As a student of GenAI (Generative Artificial Intelligence), you must be exploring the fascinating world of AI and machine learning. What specific aspects of GenAI are you interested in or currently studying?', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 20, 'total_tokens': 69, 'completion_time': 0.065333333, 'prompt_time': 0.005499023, 'queue_time': None, 'total_time': 0.070832356}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ddf2b7b-1d59-4852-bd24-9db1965de268-0', usage_metadata={'input_tokens': 20, 'output_tokens': 49, 'total_tokens': 69})

In [55]:
from langchain_core.messages import AIMessage
# trying to check if the model remebers previous messages by manually passing all the messages
llm.invoke(
    [
        HumanMessage(content="I'm Aasher, a student of GenAI"),
        AIMessage(content="Hello Aasher!\n\nWelcome to the GenAI student community! I'm excited to meet you and explore the fascinating world of Generative AI with you.\n\nWhat brings you to this field of study? Are you interested in exploring the capabilities of AI in generating human-like text, images, or music? Or perhaps you're curious about the applications of GenAI in areas such as language translation, creative writing, or even art?\n\nFeel free to share your goals, interests, or any specific aspects of GenAI that you'd like to discuss. I'm here to help and learn alongside you!"),
        HumanMessage(content="What do you know about me?")
    ]
)

AIMessage(content="I know that you're a student of GenAI, which means you're learning about Generative Artificial Intelligence. That's about the extent of my knowledge about you so far!\n\nAs we chat, I'll learn more about your interests, goals, and questions related to GenAI. I'll do my best to provide helpful information and insights to support your learning journey.\n\nSo, what's on your mind? Would you like to discuss a specific topic, ask a question, or explore a particular aspect of GenAI?", response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 154, 'total_tokens': 258, 'completion_time': 0.138666667, 'prompt_time': 0.034493862, 'queue_time': None, 'total_time': 0.173160529}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-517a855a-5dd7-460e-a9e4-2020bacea2b6-0', usage_metadata={'input_tokens': 154, 'output_tokens': 104, 'total_tokens': 258})

Now we'll use message history instead of passing all previous messages manually.
## Chat Message History

In [56]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory  # for chains

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]
# runnable with message history
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [57]:
config = {"configurable": {"session_id": "user1"}}

In [58]:
response = with_message_history.invoke(
    [HumanMessage(content="I'm Aasher, a student of GenAI")],
    config=config
)

In [59]:
from rich import print
print(response)
print(response.content)

AIMessage(
    content='Nice to meet you, Aasher! As a student of GenAI, you must be interested in the exciting field of 
Artificial Intelligence and its applications. What specific areas of GenAI are you exploring or learning about? 
Natural Language Processing, Computer Vision, Reinforcement Learning, or something else?',
    response_metadata={
        'token_usage': {
            'completion_tokens': 59,
            'prompt_tokens': 20,
            'total_tokens': 79,
            'completion_time': 0.078666667,
            'prompt_time': 0.005665163,
            'queue_time': None,
            'total_time': 0.08433183
        },
        'model_name': 'llama-3.1-8b-instant',
        'system_fingerprint': 'fp_9cb648b966',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-8e08a14c-0e16-4bbd-b555-4e00385fa15c-0',
    usage_metadata={'input_tokens': 20, 'output_tokens': 59, 'total_tokens': 79}
)

Nice to meet you, Aasher! As a student of GenAI, you must be interested in the exciting field of Artificial 
Intelligence and its applications. What specific areas of GenAI are you exploring or learning about? Natural 
Language Processing, Computer Vision, Reinforcement Learning, or something else?

In [60]:
# now invoking the same runnable and checking if it remembers our previous messages.
response = with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=config
)
response

AIMessage(content="You told me earlier that you're Aasher, a student of GenAI!", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 92, 'total_tokens': 109, 'completion_time': 0.022666667, 'prompt_time': 0.028486911, 'queue_time': None, 'total_time': 0.051153578000000005}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-59fe2ed2-c573-4d99-aabe-82594f520a88-0', usage_metadata={'input_tokens': 92, 'output_tokens': 17, 'total_tokens': 109})

In [61]:
# changing session_id
config1 = {"configurable": {"session_id": "user2"}}
# now it won't have previous messages history
response = with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=config1
)
response

AIMessage(content="Unfortunately, I don't have any information about you, so I'll have to take a wild guess. Here are a few possibilities:\n\n1. You're a human being, a unique individual with your own thoughts, feelings, and experiences.\n2. You're a conversational AI (like me!), designed to simulate human-like conversations.\n3. You're a fictional character, created by someone else's imagination.\n4. You're a person from a specific culture, country, or time period, with a rich history and identity.\n\nTo help me narrow down the possibilities, can you give me a hint or provide some context about who you are?", response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 14, 'total_tokens': 144, 'completion_time': 0.173333333, 'prompt_time': 0.00413512, 'queue_time': None, 'total_time': 0.177468453}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-805edcd9-b00b-42fd-b303-18dd922336f5-0', us

In [62]:
response = with_message_history.invoke(
    [HumanMessage(content="I'm Ahmad, a graphic designer")],
    config=config1
)
response

AIMessage(content="Nice to meet you, Ahmad!\n\nAs a graphic designer, I'm sure you're creative, detail-oriented, and skilled in visual communication. You might be working on a variety of projects, from logos and branding to print materials and digital media.\n\nCan you tell me a bit more about yourself, Ahmad? For example:\n\n* What kind of design work do you enjoy most? (e.g., UI/UX, visual identity, illustration, etc.)\n* What inspires your creative process?\n* Do you have a favorite design tool or software?\n* Are you working on any exciting projects currently?\n\nFeel free to share as much or as little as you'd like, and I'll do my best to chat with you about design, creativity, and more!", response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 160, 'total_tokens': 311, 'completion_time': 0.201333333, 'prompt_time': 0.035991677, 'queue_time': None, 'total_time': 0.23732501}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_

In [63]:
response = with_message_history.invoke(
    [HumanMessage(content="What do you know about me?")],
    config=config1
)
print(response)

AIMessage(
    content="As we've just started our conversation, I don't know much about you beyond what you've told me:\n\n1. 
Your name is Ahmad.\n2. You're a graphic designer.\n\nHowever, I can try to make some educated guesses or ask 
questions to learn more about you. Keep in mind that I'm a large language model, I don't have any personal 
knowledge or records about you, and this conversation is a fresh start.\n\nHere are a few questions to get us 
started:\n\n* Where are you from (country, city, or region)?\n* What kind of design projects have you worked on in 
the past (e.g., logos, brochures, websites, etc.)?\n* Do you have any favorite designers, artists, or creatives who
inspire you?\n* Are you working independently or as part of a design team?\n\nFeel free to answer any or all of 
these questions, and I'll do my best to learn more about you and start a conversation!",
    response_metadata={
        'token_usage': {
            'completion_tokens': 196,
            'prompt_tokens': 327,
            'total_tokens': 523,
            'completion_time': 0.261333333,
            'prompt_time': 0.072649006,
            'queue_time': None,
            'total_time': 0.333982339
        },
        'model_name': 'llama-3.1-8b-instant',
        'system_fingerprint': 'fp_9cb648b966',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-98aee62b-e06a-4bdb-80d8-5b525c6203e7-0',
    usage_metadata={'input_tokens': 327, 'output_tokens': 196, 'total_tokens': 523}
)

In [65]:
store    #it contains all session_ids and their message histories

{'user1': InMemoryChatMessageHistory(messages=[HumanMessage(content="I'm Aasher, a student of GenAI"), AIMessage(content='Nice to meet you, Aasher! As a student of GenAI, you must be interested in the exciting field of Artificial Intelligence and its applications. What specific areas of GenAI are you exploring or learning about? Natural Language Processing, Computer Vision, Reinforcement Learning, or something else?', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 20, 'total_tokens': 79, 'completion_time': 0.078666667, 'prompt_time': 0.005665163, 'queue_time': None, 'total_time': 0.08433183}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-8e08a14c-0e16-4bbd-b555-4e00385fa15c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 59, 'total_tokens': 79}), HumanMessage(content='Who am I?'), AIMessage(content="You told me earlier that you're Aasher, a student of GenAI!", respo

## Using Prompt Templates

In [68]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt  = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all questions to the best of your knowledge."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. Answer all questions to the best of your knowledge.')), MessagesPlaceholder(variable_name='messages')])

In [69]:
chain = prompt | llm

chain.invoke({"messages": [HumanMessage(content="Hi, Aasher here!")]})

AIMessage(content='Hello Aasher! Nice to meet you! Is there something I can help you with or would you like to chat?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 37, 'total_tokens': 62, 'completion_time': 0.033333333, 'prompt_time': 0.009484017, 'queue_time': None, 'total_time': 0.042817350000000004}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-96127635-af3a-4989-be20-c58d2c41428d-0', usage_metadata={'input_tokens': 37, 'output_tokens': 25, 'total_tokens': 62})

In [74]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "user3"}}

response = with_message_history.invoke(
    [HumanMessage(content="I'm Aasher. How are you?")],
    config=config
)

In [75]:
print(response)

AIMessage(
    content="Nice to meet you, Aasher! I'm doing well, thank you for asking. I'm a helpful assistant, and I'm here 
to assist you with any questions or tasks you may have. How's your day going so far?",
    response_metadata={
        'token_usage': {
            'completion_tokens': 49,
            'prompt_tokens': 91,
            'total_tokens': 140,
            'completion_time': 0.065333333,
            'prompt_time': 0.021124908,
            'queue_time': None,
            'total_time': 0.08645824099999999
        },
        'model_name': 'llama-3.1-8b-instant',
        'system_fingerprint': 'fp_f66ccb39ec',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-c812f8f3-25c8-4fae-ba25-4a4d2c8c19a4-0',
    usage_metadata={'input_tokens': 91, 'output_tokens': 49, 'total_tokens': 140}
)

In [76]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

In [77]:
response

AIMessage(content='Your name is Aasher.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 154, 'total_tokens': 161, 'completion_time': 0.009333333, 'prompt_time': 0.034805366, 'queue_time': None, 'total_time': 0.044138698999999997}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-bb2661b3-6221-414d-9d53-7bdd02705276-0', usage_metadata={'input_tokens': 154, 'output_tokens': 7, 'total_tokens': 161})

#### Let's make it more complex

In [78]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all questions to the best of your  in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | llm
chain

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='You are a helpful assistant. Answer all questions to the best of your  in {language}.')), MessagesPlaceholder(variable_name='messages')])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002758AF23B10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002758AF106D0>, model_name='llama-3.1-8b-instant', groq_api_key=SecretStr('**********'))

In [79]:
response = chain.invoke(
    {
    "messages": [HumanMessage(content="Hey there! MY name is Aasher. How are you?")],
    "language": "French",
    }
)

In [80]:
print(response.content)

Bonjour Aasher ! Je vais bien, merci ! C'est un plaisir de faire votre connaissance. Comment allez-vous aujourd'hui
? (Hello Aasher! I'm doing well, thank you! Nice to meet you. How are you today?)

#### Wrapping this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to save the chat history.

In [81]:
with_message_history = RunnableWithMessageHistory(
    chain, # adding a chain instead of llm
    get_session_history,
    input_messages_key="messages"
    )

config = {"configurable": {"session_id": "user4"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hey there! My name is Aasher. How are you?")],
        "language": "Arabic"
    },
    config=config
)


In [82]:
print(response.content)

مرحباً أشهره! (Marhaba Ashsher!) Nice to meet you! أنا بخير شكراً (Ana kheir shukraan) Thank you, I'm doing great! 
How can I assist you today, أشر؟ (Asher)?

In [83]:
# now check if it remebers the previous messages
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What's my name?")],
        "language": "Urdu"
    },
    config=config
)

In [84]:
print(response.content)

آشر (Asher) تھا! آپ کا نام آشر ہے!

## Managing the Chat History
We'll put a limit on messages given to the model so that it doesn't cross its context window

In [87]:
from langchain_core.messages import  SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",  ## keeping most recent messages
    token_counter=llm,
    include_system=True,  # system message will always be included
    allow_partial=False,  # to avoid half messages
    start_on="human"  # start from Human message

)

# dummy messages list
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, I am Khalid?"),
    AIMessage(content="Hey there, how can I help you today?"),
    HumanMessage(content="I like apples"),
    AIMessage(content="Good to know"),
    HumanMessage(content="What is 5-3?"),
    AIMessage(content="2"),
    HumanMessage(content="What is 5*3?"),
    AIMessage(content="15"),
    HumanMessage(content="thanks"),
    AIMessage(content="You're welcome"),
]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='I like apples'),
 AIMessage(content='Good to know'),
 HumanMessage(content='What is 5-3?'),
 AIMessage(content='2'),
 HumanMessage(content='What is 5*3?'),
 AIMessage(content='15'),
 HumanMessage(content='thanks'),
 AIMessage(content="You're welcome")]

**See, the two messages have been trimmed!**\
If we increase max_tokens, we can get all messages.

In [88]:
from operator import itemgetter # to retrieve specific items from a sequence or mapping object
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)
chain

RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='You are a helpful assistant. Answer all questions to the best of your  in {language}.')), MessagesPlaceholder(variable_name='messages')])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002758AF23B10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002758AF106D0>, model_name='llama-3.1-8b-instant', groq_api_key=SecretStr('**********'))

In [94]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What maths problems  did I ask?")],
        "language": "English"
    }
)

In [96]:
print(response.content)

You asked me to solve two basic math problems:

1. A subtraction problem: 5 - 3
2. A multiplication problem: 5 * 3

Let me know if you'd like me to help with anything else!

### Wrapping this in Message History

In [100]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "user6"}}

In [102]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What maths problems  did I ask?")],
        "language": "English"
    },
    config=config
)
response.content

"This is the beginning of our conversation, and you haven't asked any math problems yet. I'm here to help, though! What math question do you have for me?"